# **How to: automatically annotate problematic sequences with Poly package and the friendzymes toolkit**

Have you designed your sequence? Now it is time to remove small forbidden parts that can hinder you, not only when sequencing (e.g. hairpins, repetitive regions), but also when cloning (e.g. restriction binding sites).

What this tutorial shows is the automatic annotation of these problems. It will give you a genbank file (with these annotations attached) that you can drop into your favorite viewer, like Benchling or Snapgene.

*Tip: if it is a non-coding part of your sequence, such as promoters, RBS, etc., we recommend that you manually change the codons to synonymous codons, because these sequences generally must be kept the same to remain with their biological performances. In other words, you often will not be able to simply change the codons, but have to find other ways on a case-by-case basis.
In case you have problems with your CDS, you don't have to solve them by hand. Use the next tutorial.*




# Configurations for this tutorial




First let's run some important settings so you can run this tutorial successfully. 


Colab notebooks use python kernels to run each cell. However, because ***Poly*** is written in **Go language (golang)**, we need to install and configure some things in colab to make feasible run something in go lang here.

### **1. In order to start the golang environment, run the line below:**

In [ ]:
# this process may take a few minutes
!add-apt-repository ppa:longsleep/golang-backports -y
!apt update
!apt install golang-go
%env GOPATH=/root/go
!go get -u github.com/gopherdata/gophernotes
!cp ~/go/bin/gophernotes /usr/bin/
!npx degit gopherdata/gophernotes/kernel \
     /usr/local/share/jupyter/kernels/gophernotes

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [607 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,367

### **2. Download important data to run this tutorial**

In [ ]:
!rm -rf $GOPATH/pkg/mod/github.com/!open-!science-!global
!rm -rf $GOPATH/pkg/mod/cache/download/github.com/!open-!science-!global
!go get -u github.com/Open-Science-Global/poly@e3e1c61

In [ ]:
!wget https://raw.githubusercontent.com/Open-Science-Global/friendzymes-cookbook/main/data/anno-problematic-seqs/5_bsubori_pbs72_repa.gb

--2021-10-16 23:18:36--  https://raw.githubusercontent.com/Open-Science-Global/friendzymes-toolkit/main/data/ecoli-k12-cdss.fasta
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5054174 (4.8M) [text/plain]
Saving to: ‘ecoli-k12-cdss.fasta’

ecoli-k12-cdss.fast 100%[===================>]   4.82M  --.-KB/s    in 0.08s   

2021-10-16 23:18:37 (62.6 MB/s) - ‘ecoli-k12-cdss.fasta’ saved [5054174/5054174]



### **3. Connect Colab Notebook to your GDrive (not required)**

The previous code will download the files temporarily. If you want to download them to a folder on your drive and save it for later analysis or if you are already using this notebook to run your own files, you should connect your Google Drive to Colab and you will be able to access, read and save files permanently. 
So, if you prefer, you can do this with the code line below:



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

## After running these steps, click on **Runtime** in the menu bar & **Change Runtime Type** to Go, if it hasn't changed yet.

This will make Colab starting use a open source go kernel called Gopher Notes.

Now we are ready to work.

# **Automatic annotation of problematic sequences**

## **Importing packages and pre-requisites**

In [ ]:
package main

import (
	"fmt"
	"os"
	"path/filepath"
	"strings"
	"strconv"
	"sync"

  "github.com/Open-Science-Global/poly"
	"github.com/Open-Science-Global/poly/checks"
	"github.com/Open-Science-Global/poly/io/fasta"
	"github.com/Open-Science-Global/poly/io/genbank"
	"github.com/Open-Science-Global/poly/linearfold"
	"github.com/Open-Science-Global/poly/synthesis"
	"github.com/Open-Science-Global/poly/transform"
	"github.com/Open-Science-Global/poly/transform/codon"
	"github.com/Open-Science-Global/poly/finder"
)

## **1. Preparations for fixing problematic sequences: functions set up**


Run the cells below to generate the function to annotate problematic sequences, based on the sequences of your choice. 
For example: binding sites of restriction enzymes are problematic because they cannot be in your sequence. For iGEM teams these can be the enzymes of the Biobrick system.


**Important points:**
*   We have below ONE main function (findProblematicSequences) and 3 subfunctions (forbiddenSeqList, AvoidHairpin and homologySequencesFindProblems) that are inside the main one and also need to be set. 
*   Overall, you don't need to change anything to save and use these functions in the second step of this tutorial (2. Running), except in the first cell of code below, because note that this is OUR list of forbidden restriction enzyme binding site sequences, so for the next cell of code you will probably have to change some things. Feel free to remove some or add others.

In [ ]:
// run this to set up the subfunction 1 (REMOVE OR ADD RESTRICTIONS ENZYMES BINDING SITES AS YOU WISH)
func forbiddenSeqList() []string {
	BsaI := "GGTCTC"
	BbsI := "GAAGAC"
	BtgzI := "GCGATG"
	SapI := "GCTCTTC"
	BsmbI := "CGTCTC"
	AarI := "CACCTGC"
	PmeI := "GTTTAAAC"
	HindIII := "AAGCTT"
	PstI := "CTGCAG"
	XbaI := "TCTAGA"
	BamHI := "GGATCC"
	SmaI := "CCCGGG"
	KpnI := "GGTACC"
	SacI := "GAGCTC"
	SalI := "GTCGAC"
	EcoRI := "GAATTC"
	SphI := "GCATGC"
	AvrII := "CCTAGG"
	SwaI := "ATTTAAAT"
	AscI := "GGCGCGCC"
	FseI := "GGCCGGCC"
	PacI := "TTAATTAA"
	SpeI := "ACTAGT"
	NotI := "GCGGCCGC"
	SanDI_A := "GGGACCC"
	SanDI_T := "GGGTCCC"
	BglII := "AGATCT"
	XhoI := "CTCGAG"
	ClaI := "ATCGAT"

	List := []string{
		BsaI,
		BbsI,
		SapI,
		BsmbI,
		BtgzI,
		AarI,
		PmeI,
		HindIII,
		PstI,
		XbaI,
		BamHI,
		SmaI,
		KpnI,
		SacI,
		SalI,
		EcoRI,
		SphI,
		AvrII,
		SacI,
		SalI,
		SwaI,
		AscI,
		FseI,
		PacI,
		SpeI,
		NotI,
		SanDI_A,
		SanDI_T,
		BglII,
		XhoI,
		ClaI,
	}

	return List
}

In [ ]:
// run this to set up the subfunction 2

func AvoidHairpin(stemSize int, hairpinWindow int) func(string) []finder.Match {
	return func(sequence string) []finder.Match {
		var matches []finder.Match
		reverse := transform.ReverseComplement(sequence)
		for i := 0; i < len(sequence)-stemSize && len(sequence)-(i+hairpinWindow) >= 0; i++ {
			word := sequence[i : i+stemSize]
			rest := reverse[len(sequence)-(i+hairpinWindow) : len(sequence)-(i+stemSize)]
			if strings.Contains(rest, word) {
				location := strings.Index(rest, word)
				matches = append(matches, finder.Match{i, i + hairpinWindow - location - 1, "Harpin found in next " + strconv.Itoa(hairpinWindow) + "bp in reverse complementary sequence: " + word})
			}
		}
		return matches
	}
}

In [ ]:
// run this to set up the subfunction 3

func homologySequencesFindProblems() []string {
	// we don't have to worry about TTTTTT and GGGGGG because I already try to find also by reverse complementary of each sequence in finder
	return []string{"AAAAAA", "CCCCCC"}
}

In [ ]:
// run this to set up the MAIN function

func findProblematicSequences(sequence poly.Sequence) poly.Sequence{
	var functions []func(string) []finder.Match

	functions = append(functions, finder.ForbiddenSequence(forbiddenSeqList()))
	functions = append(functions, finder.ForbiddenSequence(homologySequencesFindProblems()))
	functions = append(functions, finder.RemoveRepeat(10))
	functions = append(functions, AvoidHairpin(20, 200))

	problems := finder.Find(strings.ToUpper(sequence.Sequence), functions)

	sequence = finder.AddMatchesToSequence(problems, sequence)
	return sequence
}


# **2. Running**


Upload your sequence file - which in this case we are using a Poly subpackage called genbank, to read files with this format.

**Important:**
* The "5_bsubori_pbs72_repa.gb" is a genbank file that has the sequence (and some annotations) of the repA replication source from Bacillus subtilis. This file can be downloaded in the settings section of this tutorial. You can change it to your own genbank file.;
* As this function is about **ANNOTATION**, here you cannot use as input a sequence in fasta format (as this is just sequence name and sequence) rather the genbank file which comes with detailed information of parts within your overall sequence.

If you don't have your sequence in a genbank file, no problem. Below we created a function called `fasta2Sequence` (run it to save it).

Then use it with your fasta file or copy/paste your sequence into it as input.

In [ ]:
// run to set up the function

func fasta2Sequence(filename string) []poly.Sequence {
  var sequences []poly.Sequence
  fastaseqs := fasta.Read(filename)
  
  for _, fastaseq := range fastaseqs {
    var sequence poly.Sequence
    var feature poly.Feature
	  sequence.Sequence = fastaseq.Sequence
    feature.Name = fastaseq.Name
	  feature.SequenceLocation.Start = 0
	  feature.SequenceLocation.End = len(sequence.Sequence)
    sequences = append(sequences, sequence)
  }
  
  return sequences
}

In [ ]:
sequences := fasta2Sequence("input.fasta")

Remember that fastas are lists of sequences, so if you have more than one in your fasta and want to analyze the first one, do so:

In [ ]:
firstSeq := findProblematicSequences(sequences[0])

// if you want the second:
secondSe2 := findProblematicSequences(sequences[1])


In case you already have your sequence in genbank, keep reading the file and proceed to find the problematic sequences:

*(This tutorial will follow with this example data, but you can modify this for your file.)*

In [ ]:
sequence := genbank.Read("5_bsubori_pbs72_repa.gb")

Run below to find the problematic sequences in your file and then save the annotated object for future inspection. **You can, for example, put this into Benchling to inspect in an easier way and be able to correct for synonymous codons if appropriate.**

In [ ]:
annotatedSequence := findProblematicSequences(sequence)
fmt.Println(annotatedSequence)

{{  0 0 0   .       {5:_BsubOri_pBS72_repA 3171 DNA  02-SEP-2021 bp false true} [] map[]}    ACATATACAGTGATGGAAGACCCGGAGTTCCTAAGGGTCTCTAAAAGAATTCAAAGTTTATACCTTACCTGGAACAAATGGTTGAAACATACGAGGCTAATATCGGCTTATTAGGAATAGTCCCTGTACTAATAAAATCAGGTGGATCAGTTGATCAGTATATTTTGGACGAAGCTCGGAAAGAATTTGGAGATGACTTGCTTAATTCCACAATTAAATTAAGGGAAAGAATAAAGCGATTTGATGTTCAAGGAATCACGGAAGAAGATACTCATGATAAAGAAGCTCTAAAACTATTCAATAACCTTACAATGGAATTGATCGAAAGGGTGGAAGGTTAATGGTACGAAAATTAGGGGATCTACCTAGAAAGCCACAAGGCGATAGGTCAAGCTTAAAGAACCCTTACATGGATCTTACAGATTCTGAAAGTAAAGAAACAACAGAGGTTAAACAAACAGAACCAAAAAGAAAAAAAGCATTGTTGAAAACAATGAAAGTTGATGTTTCAATCCATAATAAGATTAAATCGCTGCACGAAATTCTGGCAGCATCCGAAGGGAATTCATATTACTTAGAGGATACTATTGAGAGAGCTATTGATAAGATGGTTGAGACATTACCTGAGAGCCAAAAAACTTTTTATGAATATGAATTAAAAAAAAGAACCAACAAAGGCTGAGACAGACTCCAAACGAGTCTGTTTTTTTAAAAAAAATATTAGGAGCATTGAATATATATTAGAGAATTAAGAAAGACATGGGAATAAAAATATTTTAAATCCAGTAAAAATATGATAAGATTATTTCAGAATATGAAGAACTCTGTTTGTTTTTGATGAAAAAACAAACAAAAAAAATCCACCTAACGGAATCTCAATTTAACTAACAGCGGCCAAACTGAGAAG

24918 <nil>

In [ ]:
genbank.Write(annotatedSequence, "annotatedSequence.gb")